In [12]:
from medmnist import PneumoniaMNIST
import numpy as np
from medmnist import INFO, Evaluator
import matplotlib.image
import matplotlib.pyplot as plt
import tensorflow as tf
import cv2
from skimage.io import imshow, imread,imsave
from skimage.util import random_noise
import skimage.io
from skimage.transform import rescale, resize, downscale_local_mean
import dataset_without_pytorch

from dataset_without_pytorch import get_loader
import keras_tuner
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, losses
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.models import Model
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, Dropout, Conv2DTranspose, UpSampling2D, add
from keras.models import Model
from keras import regularizers

PneumoniaMNIST(split='train',download=True, size=128)

Using downloaded and verified file: /Users/christofferfuglkjaer/.medmnist/pneumoniamnist_128.npz


Dataset PneumoniaMNIST of size 128 (pneumoniamnist_128)
    Number of datapoints: 4708
    Root location: /Users/christofferfuglkjaer/.medmnist
    Split: train
    Task: binary-class
    Number of channels: 1
    Meaning of labels: {'0': 'normal', '1': 'pneumonia'}
    Number of samples: {'train': 4708, 'val': 524, 'test': 624}
    Description: The PneumoniaMNIST is based on a prior dataset of 5,856 pediatric chest X-Ray images. The task is binary-class classification of pneumonia against normal. We split the source training set with a ratio of 9:1 into training and validation set and use its source validation set as the test set. The source images are gray-scale, and their sizes are (384−2,916)×(127−2,713). We center-crop the images and resize them into 1×28×28.
    License: CC BY 4.0

In [3]:
data_flag = 'pneumoniamnist'
# data_flag = 'breastmnist'
download = True

NUM_EPOCHS = 3
BATCH_SIZE = 128
lr = 0.001

info = INFO[data_flag]
task = info['task']
n_channels = info['n_channels']
n_classes = len(info['label'])

DataClass = getattr(dataset_without_pytorch, info['python_class'])

In [4]:
# load the data
train_dataset = DataClass(split='train', download=download,size = 128)
test_dataset = DataClass(split='test', download=download,size = 128)
val_dataset= DataClass(split='val', download=download,size = 128)


# encapsulate data into dataloader form
train_loader = get_loader(dataset=train_dataset, batch_size=BATCH_SIZE)
test_loader = get_loader(dataset = test_dataset, batch_size=BATCH_SIZE)

Using downloaded and verified file: /Users/christofferfuglkjaer/.medmnist/pneumoniamnist_128.npz
Using downloaded and verified file: /Users/christofferfuglkjaer/.medmnist/pneumoniamnist_128.npz
Using downloaded and verified file: /Users/christofferfuglkjaer/.medmnist/pneumoniamnist_128.npz


In [5]:
train_file = np.load('/Users/christofferfuglkjaer/.medmnist/pneumoniamnist_128.npz')

x_train = train_file['train_images']
x_test = train_file['test_images']
x_val = train_file['val_images']
x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.
x_val=x_val.astype('float32') / 255.

print (x_val.shape)
print (x_test.shape)

(524, 128, 128)
(624, 128, 128)


In [6]:
noise_factor = 0.2

x_train_noisy = x_train + noise_factor * tf.random.normal(shape=x_train.shape)
x_test_noisy = x_test + noise_factor * tf.random.normal(shape=x_test.shape)
x_val_noisy = x_val + noise_factor * tf.random.normal(shape=x_val.shape)

x_train_noisy = tf.clip_by_value(x_train_noisy, clip_value_min=0., clip_value_max=1.)
x_test_noisy = tf.clip_by_value(x_test_noisy, clip_value_min=0., clip_value_max=1.)
x_val_noisy = tf.clip_by_value(x_val_noisy, clip_value_min=0., clip_value_max=1.)


2025-02-06 09:42:22.000117: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1 Pro
2025-02-06 09:42:22.000144: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2025-02-06 09:42:22.000148: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2025-02-06 09:42:22.000202: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-02-06 09:42:22.000237: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [ ]:
n = 10
plt.figure(figsize=(20, 2))
for i in range(n):
    ax = plt.subplot(1, n, i + 1)
    plt.title("original + noise")
    plt.imshow(tf.squeeze(x_test_noisy[i]))
    plt.gray()
plt.show()

In [ ]:
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, Dropout, Conv2DTranspose, UpSampling2D, add
from keras.models import Model
from keras import regularizers

# Encoder

n = 128
chan = 1
input_img = Input(shape=(n, n, chan))

l1 = Conv2D(32, (3, 3), padding='same', activation='relu', activity_regularizer=regularizers.l1(10e-10))(input_img)
l2 = Conv2D(32, (3, 3), padding='same', activation='relu', activity_regularizer=regularizers.l1(10e-10))(l1)
l3 = MaxPooling2D(padding='same')(l2)
#l3 = Dropout(0.3)(l3)
l4 = Conv2D(64, (3, 3),  padding='same', activation='relu', activity_regularizer=regularizers.l1(10e-10))(l3)
l5 = Conv2D(64, (3, 3), padding='same', activation='relu', activity_regularizer=regularizers.l1(10e-10))(l4)
l6 = MaxPooling2D(padding='same')(l5)
#l3 = Dropout(0.5)(l3)
l7 = Conv2D(128, (3, 3), padding='same', activation='relu', activity_regularizer=regularizers.l1(10e-10))(l6)
encoder = Model(input_img, l7)
encoder.summary()

In [ ]:
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, Dropout, Conv2DTranspose, UpSampling2D, add
from keras.models import Model
from keras import regularizers

# Encoder

n = 128
chan = 1
input_img = Input(shape=(n, n, chan))

l1 = Conv2D(32, (3, 3), padding='same', activation='relu', activity_regularizer=regularizers.l1(10e-10))(input_img)
l2 = Conv2D(32, (3, 3), padding='same', activation='relu', activity_regularizer=regularizers.l1(10e-10))(l1)
l3 = MaxPooling2D(padding='same')(l2)
#l3 = Dropout(0.3)(l3)
l4 = Conv2D(64, (3, 3),  padding='same', activation='relu', activity_regularizer=regularizers.l1(10e-10))(l3)
l5 = Conv2D(64, (3, 3), padding='same', activation='relu', activity_regularizer=regularizers.l1(10e-10))(l4)
l6 = MaxPooling2D(padding='same')(l5)
#l3 = Dropout(0.5)(l3)
l7 = Conv2D(128, (3, 3), padding='same', activation='relu', activity_regularizer=regularizers.l1(10e-10))(l6)
encoder = Model(input_img, l7)
encoder.summary()

# Decoder

l8 = UpSampling2D()(l6)

l9 = Conv2D(64, (3, 3), padding='same', activation='relu', activity_regularizer=regularizers.l1(10e-10),)(l8)
l10 = Conv2D(64, (3, 3), padding='same', activation='relu', activity_regularizer=regularizers.l1(10e-10))(l9)

l11 = add([l5, l10])
l12 = UpSampling2D()(l11)
#l3 = Dropout(0.3)(l3)
l13 = Conv2D(32, (3, 3), padding='same', activation='relu', activity_regularizer=regularizers.l1(10e-10))(l12)
l14 = Conv2D(32, (3, 3), padding='same', activation='relu', activity_regularizer=regularizers.l1(10e-10))(l13)

l15 = add([l14, l2])

# chan = 3, for RGB
decoded = Conv2D(chan, (3, 3), padding='same', activation='relu', activity_regularizer=regularizers.l1(10e-10))(l15)

# Create our network
autoencoder = Model(input_img, decoded)
# You'll understand later what this is

autoencoder.summary()

In [ ]:
autoencoder.summary()

tuner.search(x_train, y_train, epochs=5, validation_data=(x_val, y_val))
best_model = tuner.get_best_models()[0]

In [ ]:
autoencoder.compile(optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=0.0001), loss=losses.MeanSquaredError())
autoencoder.fit(x_train_noisy, x_train,epochs=25,batch_size = 16,shuffle=True, validation_data=(x_test_noisy, x_test))

In [227]:
ori_weights = autoencoder.get_weights()

In [ ]:
imshow(x_val[1])

In [ ]:
decoded_imgs=autoencoder.predict(x_val_noisy)

In [ ]:
imshow(decoded_imgs[1])

In [231]:
# encoded_imgs = autoencoder.encoder(x_test_noisy).numpy()
# decoded_imgs = autoencoder.decoder(encoded_imgs).numpy()

In [232]:
def mse(orig, res):
    return ((orig - res) ** 2).mean()
     

In [ ]:
import matplotlib.pyplot as plt



n = 5
plt.figure(figsize=(15, 5))
for i in range(n):
    # display original
    ax = plt.subplot(3, n, i + 1)
    plt.imshow(x_test[i])
    plt.title("original")
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    
    # display noisy
    ax = plt.subplot(3, n, i + 1 + n)
    plt.imshow(x_test_noisy[i])
    plt.title("noisy")
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    
    # display reconstruction
    ax = plt.subplot(3, n, i + 1 + 2 * n)
    plt.imshow(decoded_imgs[i])
    plt.title("reconstructed")
    plt.gray()
    ax.get_xaxis().set_visible(True)
    plt.xlabel(mse(x_test[i], decoded_imgs[i]))
    ax.get_yaxis().set_visible(False)

plt.show()

In [ ]:
imshow(decoded_imgs[2])

# multiple "networks"

1) 50/50 netværk 
2) 4* 25 netværk
3) 10 * 10 netværk

planen er at træne det her netværk seperat og bruge noget magic til at sætte vægtene sammen til et netværk og så evaluerer dem mod det fulde netværk 

Vi kører alle netværk ved 25 epoker. 

In [ ]:


x_train_noisy_501 = np.split(x_train_noisy,2)
x_testnoisy_501 = np.split(x_test_noisy,2)
x_train_501 = np.split(x_train,2)
x_test_501 = np.split(x_test,2)
x_val_noisy501 = np.split(x_val_noisy,2)



# 50.1

In [ ]:
n = 128
chan = 1

weights = []
epochs = 1
batchsize = 8

for i in range(len(x_train_501)):
    print(f"starter på at træne netværk {i+1}")

    input_img = Input(shape=(n, n, chan))

    l1 = Conv2D(32, (3, 3), padding='same', activation='relu', activity_regularizer=regularizers.l1(10e-10))(input_img)
    l2 = Conv2D(32, (3, 3), padding='same', activation='relu', activity_regularizer=regularizers.l1(10e-10))(l1)
    l3 = MaxPooling2D(padding='same')(l2)
    #l3 = Dropout(0.3)(l3)
    l4 = Conv2D(64, (3, 3),  padding='same', activation='relu', activity_regularizer=regularizers.l1(10e-10))(l3)
    l5 = Conv2D(64, (3, 3), padding='same', activation='relu', activity_regularizer=regularizers.l1(10e-10))(l4)
    l6 = MaxPooling2D(padding='same')(l5)
    #l3 = Dropout(0.5)(l3)
    l7 = Conv2D(128, (3, 3), padding='same', activation='relu', activity_regularizer=regularizers.l1(10e-10))(l6)
    encoder = Model(input_img, l7)


    # Decoder

    l8 = UpSampling2D()(l6)

    l9 = Conv2D(64, (3, 3), padding='same', activation='relu', activity_regularizer=regularizers.l1(10e-10),)(l8)
    l10 = Conv2D(64, (3, 3), padding='same', activation='relu', activity_regularizer=regularizers.l1(10e-10))(l9)

    l11 = add([l5, l10])
    l12 = UpSampling2D()(l11)
    #l3 = Dropout(0.3)(l3)
    l13 = Conv2D(32, (3, 3), padding='same', activation='relu', activity_regularizer=regularizers.l1(10e-10))(l12)
    l14 = Conv2D(32, (3, 3), padding='same', activation='relu', activity_regularizer=regularizers.l1(10e-10))(l13)

    l15 = add([l14, l2])


    decoded = Conv2D(chan, (3, 3), padding='same', activation='relu', activity_regularizer=regularizers.l1(10e-10))(l15)

    # Create our network
    autoencoder501 = Model(input_img, decoded)

    autoencoder501.compile(optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=0.0001), loss=losses.MeanSquaredError())
    autoencoder501.fit(x_train_noisy_501[i], x_train_501[i],epochs=50,batch_size = 8,shuffle=True, validation_data=(x_testnoisy_501[i], x_test_501[i]))
    weights.append(autoencoder501.get_weights())
    print(f"netværk {i+1} er færdig med at gøre, går videre til netværk {i+2}")



In [ ]:
decoded_imgs=autoencoder.predict(x_val_noisy501)

In [ ]:
# Ensure that the arrays have the same shape
same_shape = all(w1.shape == w2.shape for w1, w2 in zip(weights_501, weights_502))
print(same_shape)

In [ ]:
de = autoencoder501.predict(x_val)
imshow(de[0])

In [ ]:
autoencoder.evaluate(x_val)

In [266]:
weights50501 = [ori - w1 for ori, w1 in zip(ori_weights, weights_501)]
weights50502 = [ori - w2 for ori, w2 in zip(ori_weights, weights_502)]
weights5050 = [(w1 +w2)/2 + ori for w1, w2,ori in zip(weights50501, weights50502, ori_weights)]
autoencoder.set_weights(weights5050)

In [ ]:
decoded_imgs = autoencoder.predict(x_val)
imshow(decoded_imgs[3])

# 25-25



In [17]:
x_train

array([[[0.32941177, 0.34117648, 0.3529412 , ..., 0.04705882,
         0.04705882, 0.04705882],
        [0.36078432, 0.36862746, 0.3764706 , ..., 0.03921569,
         0.04313726, 0.04705882],
        [0.3764706 , 0.38431373, 0.39215687, ..., 0.03921569,
         0.04313726, 0.04313726],
        ...,
        [0.61960787, 0.627451  , 0.654902  , ..., 0.58431375,
         0.5019608 , 0.46666667],
        [0.6117647 , 0.6392157 , 0.6627451 , ..., 0.5921569 ,
         0.5019608 , 0.49019608],
        [0.61960787, 0.6392157 , 0.6627451 , ..., 0.5921569 ,
         0.49803922, 0.49803922]],

       [[0.40784314, 0.39607844, 0.39215687, ..., 0.3764706 ,
         0.36862746, 0.42352942],
        [0.41568628, 0.40784314, 0.40392157, ..., 0.39607844,
         0.3764706 , 0.40392157],
        [0.41960785, 0.43529412, 0.43529412, ..., 0.40784314,
         0.38431373, 0.38039216],
        ...,
        [0.7019608 , 0.70980394, 0.72156864, ..., 0.65882355,
         0.6431373 , 0.6313726 ],
        [0.7

In [19]:
x_train_noisy25 = np.split(x_train_noisy,4)
x_train25 = np.split(x_train,4)
x_test_noisy25 = np.split(x_test_noisy,4)
x_test25 = np.split(x_test , 4)
x_val25 = np.split(x_val_noisy,4)


In [ ]:
weights = []
epochs = 25
batchsize = 8
agg = 2
n = 128
chan = 1

for j in agg:
    for i in range(len(x_train25)):
    print(f"starter med at træne netværk {i+1}.")

    input_img = Input(shape=(n, n, chan))

    l1 = Conv2D(32, (3, 3), padding='same', activation='relu', activity_regularizer=regularizers.l1(10e-10))(input_img)
    l2 = Conv2D(32, (3, 3), padding='same', activation='relu', activity_regularizer=regularizers.l1(10e-10))(l1)
    l3 = MaxPooling2D(padding='same')(l2)
    #l3 = Dropout(0.3)(l3)
    l4 = Conv2D(64, (3, 3),  padding='same', activation='relu', activity_regularizer=regularizers.l1(10e-10))(l3)
    l5 = Conv2D(64, (3, 3), padding='same', activation='relu', activity_regularizer=regularizers.l1(10e-10))(l4)
    l6 = MaxPooling2D(padding='same')(l5)
    #l3 = Dropout(0.5)(l3)
    l7 = Conv2D(128, (3, 3), padding='same', activation='relu', activity_regularizer=regularizers.l1(10e-10))(l6)
    encoder = Model(input_img, l7)


    # Decoder

    l8 = UpSampling2D()(l6)

    l9 = Conv2D(64, (3, 3), padding='same', activation='relu', activity_regularizer=regularizers.l1(10e-10),)(l8)
    l10 = Conv2D(64, (3, 3), padding='same', activation='relu', activity_regularizer=regularizers.l1(10e-10))(l9)

    l11 = add([l5, l10])
    l12 = UpSampling2D()(l11)
    #l3 = Dropout(0.3)(l3)
    l13 = Conv2D(32, (3, 3), padding='same', activation='relu', activity_regularizer=regularizers.l1(10e-10))(l12)
    l14 = Conv2D(32, (3, 3), padding='same', activation='relu', activity_regularizer=regularizers.l1(10e-10))(l13)

    l15 = add([l14, l2])

    # chan = 3, for RGB
    decoded = Conv2D(chan, (3, 3), padding='same', activation='relu', activity_regularizer=regularizers.l1(10e-10))(l15)

    # Create our network
    autoencoder25 = Model(input_img, decoded)
    # You'll understand later what this is

    autoencoder25.compile(optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=0.0001), loss=losses.MeanSquaredError())
    autoencoder25.fit(x_train_noisy25[i], x_train25[i],epochs=epochs,batch_size = batchsize,shuffle=True, validation_data=(x_test_noisy25[i], x_test25[i]),verbose=0,)
    y = autoencoder25.get_weights()
    weights.append(y)
    print(f"netværk {i+1} er færdig med at gøre, går videre til netværk {i+2}")

starter med at træne netværk1.
netværk1 er færdig med at gøre, går videre til netværk2
starter med at træne netværk2.
netværk2 er færdig med at gøre, går videre til netværk3
starter med at træne netværk3.
netværk3 er færdig med at gøre, går videre til netværk4
starter med at træne netværk4.
netværk4 er færdig med at gøre, går videre til netværk5
